## Time Travel

When working with graphs, for understanding their decision making helps in understanding: 
- Understanding Reasoning
- Debug Mistakes
- Explore Alternatives

Langgraph provides support for Time Travel to support these usecases. Specifically you can resume the execution from the prior checkpoint id. Instead of running all the nodes in the Graph. 

In these tutorial we are going to get the chat history change the state and run the graph.

In [1]:
from langgraph.graph import StateGraph,START,END,MessagesState
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.runnables import RunnableConfig
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing_extensions import TypedDict,NotRequired

load_dotenv()

True

In [2]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", include_thoughts=False, temperature=0
)

In [3]:
class State(TypedDict):
    topic: NotRequired[str]
    joke: NotRequired[str]


In [4]:
def generate_topic(state: State) -> State:
    """LLM call to generate a topic for the joke"""
    msg = model.invoke("Give me a funny topic for a joke")
    return {"topic": msg.content}


def write_joke(state: State) -> State:
    """LLM call to write a joke based on the topic"""
    msg = model.invoke(f"Write a short joke about {state['topic']}")
    return {"joke": msg.content}

In [5]:
workflow = StateGraph(State)

# Add nodes
workflow.add_node("generate_topic", generate_topic)
workflow.add_node("write_joke", write_joke)

# Add edges to connect nodes
workflow.add_edge(START, "generate_topic")
workflow.add_edge("generate_topic", "write_joke")
workflow.add_edge("write_joke", END)

# Compile
checkpointer = InMemorySaver()
graph = workflow.compile(checkpointer=checkpointer)

Run the graph

In [6]:
import uuid


config: RunnableConfig = {"configurable": {"thread_id": uuid.uuid1()}}

In [7]:
config

{'configurable': {'thread_id': UUID('8ed777e0-cbb8-11f0-89a1-b48c9d911a92')}}

In [8]:
state = graph.invoke({}, config)
print(state)

{'topic': 'Okay, here are some funny topics for a joke, ranging from relatable to absurd:\n\n1.  **The existential dread of trying to assemble IKEA furniture.** (Universal frustration, tiny diagrams, missing screws, questioning your life choices.)\n2.  **Autocorrect fails that lead to hilariously inappropriate texts.** (Especially when texting parents or bosses.)\n3.  **Trying to explain technology to your parents/grandparents.** (The "where\'s the any key?" phenomenon, yelling at smart speakers.)\n4.  **The struggle of adulting:** bills, laundry, and pretending you know how to cook. (The sheer exhaustion of basic life tasks.)\n5.  **Your pet\'s secret life when you\'re not home.** (Are they throwing parties? Running a secret society? Judging your life choices?)\n6.  **Smart home devices that constantly misunderstand you.** (Alexa playing death metal when you asked for "smooth jazz.")\n7.  **The agony of trying to remember all your passwords.** (And the ridiculous security questions.)\

In [9]:
async for chunk in graph.astream({},config,stream_mode="updates"):
    print(chunk)

{'generate_topic': {'topic': 'Okay, here are some funny topics for a joke, ranging from relatable to absurd:\n\n1.  **The existential dread of trying to assemble IKEA furniture.** (Universal frustration, tiny diagrams, missing screws, questioning your life choices.)\n2.  **Autocorrect fails that lead to hilariously inappropriate texts.** (Especially when texting parents or bosses.)\n3.  **Trying to explain technology to your parents/grandparents.** (The "where\'s the any key?" phenomenon, yelling at smart speakers.)\n4.  **The struggle of adulting:** bills, laundry, and pretending you know how to cook. (The sheer exhaustion of basic life tasks.)\n5.  **Your pet\'s secret life when you\'re not home.** (Are they throwing parties? Running a secret society? Judging your life choices?)\n6.  **Smart home devices that constantly misunderstand you.** (Alexa playing death metal when you asked for "smooth jazz.")\n7.  **The agony of trying to remember all your passwords.** (And the ridiculous se

Going back to the history

In [10]:
from IPython.display import Markdown

states = graph.get_state_history(config)
for state in states:
    print(state.values)
    print("Next = ",state.next)
    print("checkpoint ID = ",state.config["configurable"]["checkpoint_id"])
    print("-----")

{'topic': 'Okay, here are some funny topics for a joke, ranging from relatable to absurd:\n\n1.  **The existential dread of trying to assemble IKEA furniture.** (Universal frustration, tiny diagrams, missing screws, questioning your life choices.)\n2.  **Autocorrect fails that lead to hilariously inappropriate texts.** (Especially when texting parents or bosses.)\n3.  **Trying to explain technology to your parents/grandparents.** (The "where\'s the any key?" phenomenon, yelling at smart speakers.)\n4.  **The struggle of adulting:** bills, laundry, and pretending you know how to cook. (The sheer exhaustion of basic life tasks.)\n5.  **Your pet\'s secret life when you\'re not home.** (Are they throwing parties? Running a secret society? Judging your life choices?)\n6.  **Smart home devices that constantly misunderstand you.** (Alexa playing death metal when you asked for "smooth jazz.")\n7.  **The agony of trying to remember all your passwords.** (And the ridiculous security questions.)\

Here are the checkpoint ids we got now lets take one checkpoint id to get started

In [20]:
selected_state = list(graph.get_state_history(config))[-3]

In [21]:
selected_state

StateSnapshot(values={'topic': 'Okay, here are some funny topics for a joke, ranging from relatable to absurd:\n\n1.  **The existential dread of trying to assemble IKEA furniture.** (Universal frustration, tiny diagrams, missing screws, questioning your life choices.)\n2.  **Autocorrect fails that lead to hilariously inappropriate texts.** (Especially when texting parents or bosses.)\n3.  **Trying to explain technology to your parents/grandparents.** (The "where\'s the any key?" phenomenon, yelling at smart speakers.)\n4.  **The struggle of adulting:** bills, laundry, and pretending you know how to cook. (The sheer exhaustion of basic life tasks.)\n5.  **Your pet\'s secret life when you\'re not home.** (Are they throwing parties? Running a secret society? Judging your life choices?)\n6.  **Smart home devices that constantly misunderstand you.** (Alexa playing death metal when you asked for "smooth jazz.")\n7.  **The agony of trying to remember all your passwords.** (And the ridiculous 

Update the state of the selected state to run without rerunning the entire nodes 

In [22]:
new_config= graph.update_state(selected_state.config,values={"topic":"computer"})
new_config

{'configurable': {'thread_id': '8ed777e0-cbb8-11f0-89a1-b48c9d911a92',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0cbb97-bc56-6bdc-8002-de08ba5d8cb6'}}

**Note:** In case if you want to update the messages you need to send the exact `id` to update the message.

Resume the execution from the checkpoint. By Passing the New config to it. But it must contain  the same thread_id

In [23]:
async for chunk in graph.astream(None,new_config,stream_mode="updates"):
    print(chunk)

{'write_joke': {'joke': 'Why was the computer cold?\n\nBecause it left its Windows open!'}}
